In [1]:
import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

#time
from datetime import datetime
from datetime import timedelta

import jieba
import jieba.analyse
jieba.set_dictionary('dict.idkrsi.txt')     # 改預設字典
jieba.analyse.set_stop_words("stopword.goatwang.kang.txt") #指定stopwords字典

# get data
# ! conda install pandas-datareader s
#import pandas_datareader as pdr

# visual
# ! pip install mpl-finance
#import matplotlib.pyplot as plt
#import mpl_finance as mpf

#import seaborn as sns


# https://github.com/mrjbq7/ta-lib
# ! pip install ta-lib
#import talib

In [2]:
df_bbs = pd.read_csv("bda2019_dataset/bbs2.csv",encoding="utf-8")
df_forum = pd.read_csv("bda2019_dataset/forum2.csv",encoding="utf-8")
df_news = pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
df_news['comment_count']=0

df_article = pd.concat([df_forum, df_bbs, df_news])  #三個合併
del df_bbs, df_forum, df_news

df_article['post_time'] = pd.to_datetime(df_article['post_time'])
df_article['post_time2'] = df_article['post_time'].dt.date   # .dt.date用在dataframe  .date()用在一個 #只留日期
#df_article['label'] = 'even'
df_article['content'] = df_article['content'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article['title'] = df_article['title'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article = df_article.sort_values(by=['post_time']).reset_index(drop=True) # 用post_time排序 # 在重設index
df_article.head(2)
#df_article2 = df_article[['post_time2','title','content']]

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
0,鉅亨網,0,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,1451580107540_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 00:06:00,重大要聞,yahoo股市,淪落的2015 竟是78年來最難獲利的一年,2016-01-01
1,Bokolo,10,1.原文連結(必須檢附)： http://magazine.chinatimes.com/m...,1451580710262_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1451580509.A.722...,2016-01-01 00:48:25,Stock,Ptt,[新聞] 第三方支付 台灣普及關鍵年,2016-01-01


In [3]:
df_TWSE2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv",encoding="utf-8")
df_TWSE2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv",encoding="utf-8")
df_TWSE2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv",encoding="utf-8")

df_TWSE = pd.concat([df_TWSE2016, df_TWSE2017, df_TWSE2018])  #三年合併
del df_TWSE2016, df_TWSE2017, df_TWSE2018

# ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '成交值(千元)', '成交筆數(筆)', '流通在外股數(千股)', '本益比-TSE', '股價淨值比-TSE']
df_TWSE['證券代碼'] = df_TWSE['證券代碼'].astype(str)
df_TWSE['年月日'] = pd.to_datetime(df_TWSE['年月日'])
df_TWSE['開盤價(元)'] = df_TWSE['開盤價(元)'].str.replace(',' , '').astype('float64') # 1,000 to 1000 to float
df_TWSE['最高價(元)'] = df_TWSE['最高價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['最低價(元)'] = df_TWSE['最低價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['收盤價(元)'] = df_TWSE['收盤價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交量(千股)'] = df_TWSE['成交量(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['成交值(千元)'] = df_TWSE['成交值(千元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交筆數(筆)'] = df_TWSE['成交筆數(筆)'].str.replace(',' , '').astype('int64')
df_TWSE['流通在外股數(千股)'] = df_TWSE['流通在外股數(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['本益比-TSE'] = df_TWSE['本益比-TSE'].str.replace(',' , '').astype('float64')
df_TWSE['股價淨值比-TSE'] = df_TWSE['股價淨值比-TSE'].astype('float64')
df_TWSE.head(2) 

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,1101 台泥,2016-12-30,29.09,29.64,28.93,29.64,6121.0,213121.0,2939,3692175.0,20.56,1.23
1,1102 亞泥,2016-12-30,24.91,24.91,24.67,24.81,3347.0,88078.0,1227,3361447.0,31.75,0.73


In [4]:
# 選那家股票
#company_name = '國巨'
company_name = '奇力新'


# 文章包含那家字
#company_words = '被動元件|積層陶瓷電容|MLCC|電感|晶片電阻|車用電子|凱美|同欣電|大毅|君耀|普斯|國巨'
company_words = '被動元件|積層陶瓷電容|MLCC|電感|晶片電阻|車用電子|飛磁|旺詮|美磊|美桀|向華科技|奇力新'

# 漲跌幾%
PA = 0.05

# even幾%
PAE = 0.003

In [5]:
# 用日期排序 再把index重排
#2327 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('國巨')].sort_values(by=['年月日']).reset_index(drop=True)
#2456 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('奇力新')].sort_values(by=['年月日']).reset_index(drop=True)
#2478 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('大毅')].sort_values(by=['年月日']).reset_index(drop=True)
#6271 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('同欣電')].sort_values(by=['年月日']).reset_index(drop=True)

df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains(company_name)].sort_values(by=['年月日']).reset_index(drop=True)
del df_TWSE
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46


In [6]:
##增欄位:fluctuation幅度 tag漲跌平
df_trend['fluctuation'] = 0.0
df_trend['tag']='--'
df_trend['closeshift'] = 0.0
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,0.0,--,0.0
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.0,--,0.0


In [7]:
# ##增欄位:fluctuation幅度 tag漲跌平
# df_trend['fluctuation'] = 0.0
# df_trend['tag']='--'

# ###計算漲跌
# for index, row in df_trend.iterrows():
#     try:
#         margin =(float(df_trend.loc[index,'收盤價(元)']) - float(df_trend.loc[index-1,'收盤價(元)']) )/ float(df_trend.loc[index-1,'收盤價(元)'])
#         df_trend.loc[index,'fluctuation']=margin
#         if margin >=0.03:
#             df_trend.loc[index,'tag']='up'
#         elif margin <= -0.03:
#             df_trend.loc[index,'tag']='down'
#         else:
#             df_trend.loc[index,'tag']='even'
#     except:
#         continue

In [8]:
df_trend['closeshift'] = df_trend['收盤價(元)'].shift(periods=1)#.fillna(value=0.0, inplace=True)
#df_trend['closeshift'].fillna(value= 0.0, inplace=True)
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,0.0,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.0,--,80.2


In [9]:
df_trend['fluctuation'] = (df_trend['收盤價(元)'] - df_trend['closeshift']) / df_trend['closeshift']
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,NaN,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.060599,--,80.2


In [13]:
print('fluctuation std = ',df_trend['fluctuation'].std(axis=0))
print('fluctuation mean = ',df_trend['fluctuation'].mean(axis=0))

fluctuation std =  0.030403952190342375
fluctuation mean =  0.0004319276970365839


In [10]:
df_trend.loc[df_trend['fluctuation'] >=  PA, 'tag'] = 'up'
df_trend.loc[df_trend['fluctuation'] <= -PA, 'tag'] = 'down'
df_trend.loc[(df_trend['fluctuation'] >= -PAE) & (df_trend['fluctuation'] <= PAE), 'tag'] = 'even'
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,NaN,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.060599,up,80.2


In [11]:
len(df_trend[df_trend['tag']=='up'])

37

In [12]:
len(df_trend[df_trend['tag']=='down'])

39

In [13]:
len(df_trend[df_trend['tag']=='even'])

90

In [14]:
#df_company = df_article[ df_article['content'].str.contains('國巨')]   # df 某欄位 string contains "國巨"
#df_company = df_article[ df_article['content'].str.contains('奇力新')] 
#df_company = df_article[ df_article['content'].str.contains('大毅')]  
#df_company = df_article[ df_article['content'].str.contains('同欣電 ')]
df_company = df_article[ df_article['content'].str.contains(company_words)]   
print(len(df_company))
del df_article
df_company.head(2)

10553


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
30,時報資訊,0,台股2015年下半年跌勢洶洶，全年跌幅達10.14％，但12檔具利多題材個股，股價多頭氣盛，...,1451618809219_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 10:59:00,重大要聞,yahoo股市,《熱門族群》12檔話題當紅股，完美封關,2016-01-01
97,鉅亨網,0,鉅亨網記者王莞甯 台北台股2015年封關日最後一檔有神秘買盤進場，強拉加權指數上漲39點，元...,1451715108530_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-02 13:50:00,重大要聞,yahoo股市,錢冠州：總統大選不再是大變數 元月行情伺機而動！,2016-01-02


In [15]:
stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [16]:
# 'content'全部切詞
corpus = []  # array
for index, row in df_company.iterrows():  
    not_cut = df_company.loc[index,'content']
#     not_cut = row['description']                    # 跟上一行一樣意思
    seg_generator = jieba.cut(not_cut, cut_all=False)  # genarator
    seglist = list(seg_generator)                     # 整篇文章string切出來的list
#    seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟
    corpus.append(' '.join(seglist))          # ' '.join(seg_generator)也可

df_company["content2"]=corpus  

Building prefix dict from /home/barry/projects/stock/dict.idkrsi.txt ...
Loading model from cache /tmp/jieba.ufc1c7fa02eb193cee2bb78b34d63c017.cache
Loading model cost 0.426 seconds.
Prefix dict has been built succesfully.


In [17]:
df_company.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
30,時報資訊,0,台股2015年下半年跌勢洶洶，全年跌幅達10.14％，但12檔具利多題材個股，股價多頭氣盛，...,1451618809219_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 10:59:00,重大要聞,yahoo股市,《熱門族群》12檔話題當紅股，完美封關,2016-01-01,台股 2015 年 下半年 跌勢 洶洶 ， 全年 跌幅 達 10.14 ％ ， 但 12 檔...
97,鉅亨網,0,鉅亨網記者王莞甯 台北台股2015年封關日最後一檔有神秘買盤進場，強拉加權指數上漲39點，元...,1451715108530_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-02 13:50:00,重大要聞,yahoo股市,錢冠州：總統大選不再是大變數 元月行情伺機而動！,2016-01-02,鉅亨網 記者 王莞 甯 台北 台股 2015 年 封關日 最後 一檔 有 神秘 買盤 進...


In [18]:
df_trend.loc[2,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[1,'年月日'].date()

True

In [19]:
df_trend.loc[5,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[4,'年月日'].date()

False

In [20]:
d = df_trend.loc[1,'年月日'].date() - df_trend.loc[ 1-1 ,'年月日'].date() #相減差幾天
d

datetime.timedelta(1)

In [21]:
d.days #只取天數

1

In [22]:
int(d.days) #幾天 轉整數

1

In [23]:
df_trend.loc[3,'年月日'].date() 

datetime.date(2016, 1, 7)

In [24]:
df_company[ df_company['post_time2'] == df_trend.loc[3,'年月日'].date() ].head() # 某欄位 == n 的 全部撈出來

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
999,Moneydj理財網,0,1.週三台股在美股重挫以及陸股再度啟動融斷機制之下，指數重挫點，指數跌破十年線，所幸尾盤大型...,1452156767683_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 16:29:00,重大要聞,yahoo股市,日盛投顧：台股短線有反彈機會,2016-01-07,1 . 週三 台股 在 美股 重挫 以及 陸股 再度 啟動 融斷 機制 之下 ， 指數 重挫...


In [25]:
# # 演算法 
# for index, row in df_2327.iterrows():
#     try:
#         if df_2327.loc[index,'年月日'].date() +  timedelta(days=-1) == df_2327.loc[index-1,'年月日'].date():                        
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差1天  # 那前1天的文章標上當天的漲跌   
        
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-2) == df_2327.loc[index-1,'年月日'].date():           
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差2天 #那前2天的文章標上當天的漲跌
            
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-3) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-4) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-4), 'label'] = df_2327.loc[index,'tag']


#     except:
#         continue

In [26]:
# 看所有相差的天數
# for index, row in df_2327.iterrows():
#     try:
#         n = df_2327.loc[index,'年月日'].date() - df_2327.loc[index-1,'年月日'].date()
#         print(n)
#     except:
#         continue

# 最多12天

In [27]:
# 如果股票前一筆差n天  # 那前n天的文章標上當天的漲跌 
df_company['label5566']='--'
for index, row in df_trend.iterrows():
    try:
        n = int((df_trend.loc[index,'年月日'].date() - df_trend.loc[index-1,'年月日'].date()).days ) # 差幾個datetime # 轉天數 # 再轉整數
#        print(n)

        for i in range(1, n+1):
#            print(i)
            df_company.loc[df_company['post_time2'] ==  df_trend.loc[index,'年月日'].date() +  timedelta(days=-i), 'label5566'] = df_trend.loc[index,'tag']
    except:
        continue

In [28]:
print(len(df_company[df_company['label5566']=='down']))
df_company[df_company['label5566']=='down'].head(2)

1284


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
1522,財訊快報,0,◆全台勞檢啟動，11類行業小心，其中284家電子零組件企業，實施全面普查。<BR> ◆海外分...,1452472319234_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-11 08:05:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-11,◆ 全台 勞檢 啟動 ， 11 類 行業 小心 ， 其中 284 家 電子 零組件 企業 ，...,down
1530,時報資訊,0,陸股暫停熔斷機制，減緩市場對於系統性風險的疑慮，法人指出，台股短線壓力仍大，若結構出現惡化現...,1452472329341_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-11 08:15:00,重大要聞,yahoo股市,《基金》全球經濟趨緩，慎選台股標的,2016-01-11,陸股 暫停 熔斷 機制 ， 減緩 市場 對於 系統性 風險 的 疑慮 ， 法人 指出 ， 台...,down


In [29]:
print(len(df_company[df_company['label5566']=='up']))
df_company[df_company['label5566']=='up'].head(2)

891


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
222,財訊快報,0,◆千億科技預算，張善政喊加碼，主計總處初步匡列1 061億、年增3%，可能再調高。<BR> ...,1451867210012_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 07:56:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-04,◆ 千億 科技 預算 ， 張善政 喊 加碼 ， 主計 總處 初步 匡列 1 061 億 ...,up
289,時報資訊,0,2016年開紅盤日，台股狂洩逾200點！儘管2015年最後一個交易日拉尾盤，守住8300點，...,1451879808470_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 11:34:00,重大要聞,yahoo股市,《盤中解析》紅盤日變臉，台股狂洩逾200點,2016-01-04,2016 年開 紅盤 日 ， 台股 狂洩 逾 200 點 ！ 儘管 2015 年 最後 一個...,up


In [30]:
print(len(df_company[df_company['label5566']=='even']))
df_company[df_company['label5566']=='even'].head(2)

911


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
3121,鉅亨網,0,受到月線反壓、外資籌碼面持續偏保守的影響下，台股期現貨今(20)日同步下壓，加權指數開低走低...,1453258680867_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-20 10:42:00,重大要聞,yahoo股市,台股盤中─個股漲勢稀落 指數一度大跌百點 台指期難拉高結算,2016-01-20,受到 月線 反壓 、 外資 籌碼面 持續 偏 保守 的 影響 下 ， 台股 期 現貨 今 (...,even
3192,Moneydj理財網,0,1.歐美股市走穩，不過受期指結算影響，台股週三開低，加上港股重挫，指數震盪走低，盤面上部份生...,1453281180692_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-20 16:16:00,重大要聞,yahoo股市,日盛投顧：台股短線參考區間7600-7900,2016-01-20,1 . 歐美 股市 走穩 ， 不過 受 期指 結算 影響 ， 台股 週三 開低 ， 加上 港...,even


In [31]:
#df_company2 = df_company[df_company['label5566'].str.contains('up|down|even')]
#df_company2.to_csv('5pa.csv')

In [32]:
import re
features = []   # features=list()
with open('finance.words.txt', 'r',encoding='utf-8') as data:
    for line in data:
#        line = re.sub('[a-zA-Z0-9\W]', '', line)                     # 把數字英文去掉
        line = re.sub('[0-9]', '', line)                              # 把數字去掉
        features.append(line.replace('\n', '').replace(' ', ''))      # 空格 \n去掉

print(len(features))
print(type(features))
features[:10]

3405
<class 'list'>


['營收', '交易', '成長', '上漲', '外資', '法人', '增加', '下跌', '獲利', '股']

In [33]:
#df_keyword1 = pd.read_csv("final_higher_tf_idf_part.csv",encoding="utf-8") #上漲形容詞
#df_keyword2 = pd.read_csv("final_lower_tf_idf_part.csv",encoding="utf-8")  #下跌形容詞
#df_keyword = pd.concat([df_keyword1,df_keyword2])
#del df_keyword1,df_keyword2
#df_keyword.head()

In [34]:
#features = df_keyword['key'].to_numpy()
#features

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

#features = [ '上漲','下跌','看好','走高','走低','漲停','跌停']
#features = features[:1000]
#cv = TfidfVectorizer()   #預設有空格就一個feature
cv = TfidfVectorizer(vocabulary = features) # 設定自己要的詞 

r = pd.SparseDataFrame(cv.fit_transform(df_company['content2']), 
                       df_company.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)

r.fillna(value=0.0, inplace=True)
r.head(2)

,營收,交易,成長,上漲,外資,法人,增加,下跌,獲利,股,...,跌停價,死魚,抬轎,高殖利率股,不弱,有守,續貶,跌回,轉空,執行率
30,0.0,0.0,0.0,0.056979,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0.0,0.0,0.0,0.042293,0.036978,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
# from sklearn.feature_extraction.text import CountVectorizer

# #features = [ '上漲','下跌','看好','走高','走低','漲停','跌停']
# #features = features[:1000]
# #cv = CountVectorizer()   #預設有空格就一個feature
# cv = CountVectorizer(vocabulary = features) # 設定自己要的詞 

# r = pd.SparseDataFrame(cv.fit_transform(df_company['content2']), 
#                        df_company.index,
#                        cv.get_feature_names(), 
#                        default_fill_value=0)

# r.head(2)

In [37]:
df_company2 = pd.concat([df_company,r], axis=1)
df_company2.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,...,跌停價,死魚,抬轎,高殖利率股,不弱,有守,續貶,跌回,轉空,執行率
30,時報資訊,0,台股2015年下半年跌勢洶洶，全年跌幅達10.14％，但12檔具利多題材個股，股價多頭氣盛，...,1451618809219_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 10:59:00,重大要聞,yahoo股市,《熱門族群》12檔話題當紅股，完美封關,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,鉅亨網,0,鉅亨網記者王莞甯 台北台股2015年封關日最後一檔有神秘買盤進場，強拉加權指數上漲39點，元...,1451715108530_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-02 13:50:00,重大要聞,yahoo股市,錢冠州：總統大選不再是大變數 元月行情伺機而動！,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
df_company2 = df_company2[df_company2['label5566'].str.contains('up|down')]   #只取漲跌
Y = df_company2['label5566']
X = df_company2[features]
X.fillna(value=0.0, inplace=True)
X.head(2)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,營收,交易,成長,上漲,外資,法人,增加,下跌,獲利,股,...,跌停價,死魚,抬轎,高殖利率股,不弱,有守,續貶,跌回,轉空,執行率
222,0.035882,0.057439,0.114125,0.0,0.032471,0.000000,0.0,0.041307,0.039377,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0
289,0.000000,0.000000,0.000000,0.0,0.070488,0.048611,0.0,0.239114,0.028493,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.08156,0.0,0.0,0.0


In [39]:
Y.head(2)

222    up
289    up
Name: label5566, dtype: object

In [40]:
X = X.to_numpy()
Y = Y.to_numpy()

In [41]:
#將X:features array, Y:lable array 都切成 1:4

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)
print(len(X_train),len(X_validation))

1740 435


In [42]:
model_RandomForest = RandomForestClassifier()
model_RandomForest.fit(X_train, Y_train)
print(model_RandomForest.score(X_train, Y_train))

predictions = model_RandomForest.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9833333333333333
0.664367816091954
[[224  39]
 [107  65]]
              precision    recall  f1-score   support

        down       0.68      0.85      0.75       263
          up       0.62      0.38      0.47       172

   micro avg       0.66      0.66      0.66       435
   macro avg       0.65      0.61      0.61       435
weighted avg       0.66      0.66      0.64       435



/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [43]:
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_XGBClassifier = XGBClassifier()
name = 'XGBClassifier'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_XGBClassifier, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

[0.6408046  0.65517241 0.66091954 0.6408046  0.64942529]
XGBClassifier: 0.649425 (0.007922)


In [44]:
model_XGBClassifier = XGBClassifier()
model_XGBClassifier.fit(X_train, Y_train)
print(model_XGBClassifier.score(X_train, Y_train))

predictions = model_XGBClassifier.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.8419540229885057
0.7034482758620689
[[231  32]
 [ 97  75]]
              precision    recall  f1-score   support

        down       0.70      0.88      0.78       263
          up       0.70      0.44      0.54       172

   micro avg       0.70      0.70      0.70       435
   macro avg       0.70      0.66      0.66       435
weighted avg       0.70      0.70      0.69       435



In [45]:
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_GradientBoost = GradientBoostingClassifier()
name = 'GradientBoost'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_GradientBoost, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

[0.62931034 0.6408046  0.65804598 0.64655172 0.65517241]
GradientBoost: 0.645977 (0.010345)


In [46]:
model_GradientBoost = GradientBoostingClassifier()
model_GradientBoost.fit(X_train, Y_train)
print(model_GradientBoost.score(X_train, Y_train))

predictions = model_GradientBoost.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.8563218390804598
0.7103448275862069
[[232  31]
 [ 95  77]]
              precision    recall  f1-score   support

        down       0.71      0.88      0.79       263
          up       0.71      0.45      0.55       172

   micro avg       0.71      0.71      0.71       435
   macro avg       0.71      0.66      0.67       435
weighted avg       0.71      0.71      0.69       435



In [47]:
model_DecisionTree = DecisionTreeClassifier()
model_DecisionTree.fit(X_train, Y_train)
print(model_DecisionTree.score(X_train, Y_train))

predictions = model_DecisionTree.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9994252873563219
0.639080459770115
[[188  75]
 [ 82  90]]
              precision    recall  f1-score   support

        down       0.70      0.71      0.71       263
          up       0.55      0.52      0.53       172

   micro avg       0.64      0.64      0.64       435
   macro avg       0.62      0.62      0.62       435
weighted avg       0.64      0.64      0.64       435



In [48]:
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_KNeighbors = KNeighborsClassifier()
name = 'KNN'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_KNeighbors, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

[0.64942529 0.62643678 0.67528736 0.60344828 0.6637931 ]
KNN: 0.643678 (0.025894)


In [49]:
model_KNeighbors = KNeighborsClassifier()
model_KNeighbors.fit(X_train, Y_train)
print(model_KNeighbors.score(X_train, Y_train))

predictions = model_KNeighbors.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.7954022988505747
0.7057471264367816
[[214  49]
 [ 79  93]]
              precision    recall  f1-score   support

        down       0.73      0.81      0.77       263
          up       0.65      0.54      0.59       172

   micro avg       0.71      0.71      0.71       435
   macro avg       0.69      0.68      0.68       435
weighted avg       0.70      0.71      0.70       435



In [50]:
model_GaussianNB = GaussianNB()
model_GaussianNB.fit(X_train, Y_train)
print(model_GaussianNB.score(X_train, Y_train))

predictions = model_GaussianNB.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.8281609195402299
0.5793103448275863
[[126 137]
 [ 46 126]]
              precision    recall  f1-score   support

        down       0.73      0.48      0.58       263
          up       0.48      0.73      0.58       172

   micro avg       0.58      0.58      0.58       435
   macro avg       0.61      0.61      0.58       435
weighted avg       0.63      0.58      0.58       435



In [51]:
model_MultinomialNB = MultinomialNB()
model_MultinomialNB.fit(X_train, Y_train)
print(model_MultinomialNB.score(X_train, Y_train) )

predictions = model_MultinomialNB.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_MultinomialNB.coef_)

0.714367816091954
0.6344827586206897
[[249  14]
 [145  27]]
              precision    recall  f1-score   support

        down       0.63      0.95      0.76       263
          up       0.66      0.16      0.25       172

   micro avg       0.63      0.63      0.63       435
   macro avg       0.65      0.55      0.51       435
weighted avg       0.64      0.63      0.56       435

[[-5.64852674 -6.41901294 -6.02619808 ... -8.55935314 -8.66259533
  -8.97402661]]


In [52]:
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_LogisticRegression = LogisticRegression()
name = 'MLP'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_LogisticRegression, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

[0.65229885 0.65804598 0.67528736 0.66954023 0.67241379]
MLP: 0.665517 (0.008829)


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/lo

In [53]:
model_LogisticRegression = LogisticRegression()
model_LogisticRegression.fit(X_train, Y_train)
print(model_LogisticRegression.score(X_train, Y_train))

predictions = model_LogisticRegression.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_LogisticRegression.coef_)

0.8333333333333334
0.7011494252873564
[[229  34]
 [ 96  76]]
              precision    recall  f1-score   support

        down       0.70      0.87      0.78       263
          up       0.69      0.44      0.54       172

   micro avg       0.70      0.70      0.70       435
   macro avg       0.70      0.66      0.66       435
weighted avg       0.70      0.70      0.68       435

[[-1.17222418  0.43621    -0.06713002 ... -0.05860779  0.04448703
  -0.25976489]]


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [54]:
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_svclinear = SVC(kernel='linear')
name = 'SVC'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_svclinear, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

[0.66091954 0.6637931  0.71551724 0.68103448 0.68678161]
SVC: 0.681609 (0.019608)


In [55]:
model_svclinear = SVC(kernel='linear')
model_svclinear.fit(X_train, Y_train)
print(model_svclinear.score(X_train, Y_train))
predictions = model_svclinear.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_svclinear.coef_)

0.8747126436781609
0.7172413793103448
[[221  42]
 [ 81  91]]
              precision    recall  f1-score   support

        down       0.73      0.84      0.78       263
          up       0.68      0.53      0.60       172

   micro avg       0.72      0.72      0.72       435
   macro avg       0.71      0.68      0.69       435
weighted avg       0.71      0.72      0.71       435

[[-0.76375041  0.72612338 -0.09042225 ... -0.02918578  0.08326023
  -0.1549043 ]]


In [56]:
from sklearn.svm import LinearSVC
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_LinearSVC = LinearSVC()
#model_LinearSVC = LinearSVC(loss='hinge')
name = 'SVC'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_LinearSVC, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))


[0.67528736 0.66954023 0.7183908  0.68390805 0.69827586]
SVC: 0.689080 (0.017564)


In [57]:
model_LinearSVC = LinearSVC()
model_LinearSVC.fit(X_train, Y_train)
print(model_LinearSVC.score(X_train, Y_train))
predictions = model_LinearSVC.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_LinearSVC.coef_)

0.946551724137931
0.6988505747126437
[[206  57]
 [ 74  98]]
              precision    recall  f1-score   support

        down       0.74      0.78      0.76       263
          up       0.63      0.57      0.60       172

   micro avg       0.70      0.70      0.70       435
   macro avg       0.68      0.68      0.68       435
weighted avg       0.69      0.70      0.70       435

[[-0.58117597  0.37502201  0.17897764 ... -0.13591992 -0.05167275
  -0.28588632]]


In [58]:
from sklearn.linear_model import SGDClassifier

# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_SGDClassifier = SGDClassifier(loss='hinge')
#model_SGDClassifier = SGDClassifier(loss='log')
name = 'SVC'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_SGDClassifier, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDCla

[0.5862069  0.58908046 0.6637931  0.68390805 0.6637931 ]
SVC: 0.637356 (0.041259)


In [59]:
model_SGDClassifier = SGDClassifier(loss='hinge')
model_SGDClassifier.fit(X_train, Y_train)
print(model_SGDClassifier.score(X_train, Y_train))
predictions = model_SGDClassifier.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_SGDClassifier.coef_)

0.8764367816091954
0.6666666666666666
[[154 109]
 [ 36 136]]
              precision    recall  f1-score   support

        down       0.81      0.59      0.68       263
          up       0.56      0.79      0.65       172

   micro avg       0.67      0.67      0.67       435
   macro avg       0.68      0.69      0.67       435
weighted avg       0.71      0.67      0.67       435

[[-1.35674754  1.1892719   0.13507847 ... -1.14692278  0.05760463
  -0.59799342]]


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [60]:
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_MLP = MLPClassifier(hidden_layer_sizes=(256,256, ),max_iter=256)
name = 'MLP'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_MLP, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

[0.67241379 0.68678161 0.74712644 0.68103448 0.70689655]
MLP: 0.698851 (0.026673)


In [61]:
model_MLP = MLPClassifier(hidden_layer_sizes=(256, 256,), max_iter=256)
model_MLP.fit(X_train, Y_train)
print(model_MLP.score(X_train, Y_train))

predictions = model_MLP.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_MLP.coefs_)

0.9994252873563219
0.728735632183908
[[207  56]
 [ 62 110]]
              precision    recall  f1-score   support

        down       0.77      0.79      0.78       263
          up       0.66      0.64      0.65       172

   micro avg       0.73      0.73      0.73       435
   macro avg       0.72      0.71      0.71       435
weighted avg       0.73      0.73      0.73       435

[array([[-0.01250838,  0.0085941 ,  0.03364121, ...,  0.03531847,
        -0.00995161,  0.00310071],
       [-0.0108183 , -0.00434747,  0.02452052, ...,  0.00041447,
         0.02549074, -0.01718165],
       [-0.04280206,  0.0093778 , -0.05259625, ...,  0.00778808,
         0.06609891, -0.02884467],
       ...,
       [ 0.00215422, -0.02496801,  0.03072602, ...,  0.06344268,
        -0.02041774, -0.02684463],
       [-0.0139515 ,  0.00111965,  0.01726507, ...,  0.0379245 ,
        -0.00352183, -0.00475659],
       [-0.06847205, -0.04683234,  0.04535596, ...,  0.04884753,
        -0.02402444, -0.03863771]])

In [62]:
#df_company2.to_csv('5pa.csv')